In [1]:
# Load packages
import os
import pandas as pd
import numpy as np
 # Required for basic python plotting functionality
import matplotlib.pyplot as plt
# Required for formatting dates later in the case
import datetime
import matplotlib.dates as mdates
# Advanced plotting functionality with seaborn
import seaborn as sns

sns.set(style="whitegrid")  # can set style depending on how you'd like it to look
# Load the data into python
energy_consumption_file = os.path.join(os.getcwd(), "./districts_info.csv")
# electricity_generation_file = os.path.join(
#     os.getcwd(), "data/electricity_generation.csv"
# )
# print(energy_consumption_file)
energy_df = pd.read_csv(energy_consumption_file)

temp1 = os.path.join(os.getcwd(), "./engagement_data/1000.csv")
# electricity_generation_file = os.path.join(
#     os.getcwd(), "data/electricity_generation.csv"
# )
# print(energy_consumption_file)
otherDF = pd.read_csv(temp1)
# electricity_df = pd.read_csv(electricity_generation_file)

In [ ]:
# Load packages
import os
import pandas as pd
import numpy as np
 # Required for basic python plotting functionality
import matplotlib.pyplot as plt
# Required for formatting dates later in the case
import datetime
import matplotlib.dates as mdates
# Advanced plotting functionality with seaborn
import seaborn as sns

sns.set(style="whitegrid")  # can set style depending on how you'd like it to look
# Load the data into python
energy_consumption_file = os.path.join(os.getcwd(), "./districts_info.csv")
# electricity_generation_file = os.path.join(
#     os.getcwd(), "data/electricity_generation.csv"
# )
# print(energy_consumption_file)
energy_df = pd.read_csv(energy_consumption_file)

temp1 = os.path.join(os.getcwd(), "./engagement_data/1000.csv")
# electricity_generation_file = os.path.join(
#     os.getcwd(), "data/electricity_generation.csv"
# )
# print(energy_consumption_file)
otherDF = pd.read_csv(temp1)
# electricity_df = pd.read_csv(electricity_generation_file)

In [2]:
disctrictFilesPath = os.path.join(os.getcwd(), "./districts_info.csv")

discritDF = pd.read_csv(disctrictFilesPath)

engagmentDF = os.path.join(os.getcwd(), "./engagement_data/1000.csv")

engagmentDF = pd.read_csv(engagmentDF)

productInfoDF = os.path.join(os.getcwd(), "./products_info.csv")

productInfoDF = pd.read_csv(productInfoDF)

In [3]:
discritDF.shape
discritDF.head()

,district_id,state,locale,pct_black/hispanic,pct_free/reduced,county_connections_ratio,pp_total_raw
0,8815,Illinois,Suburb,"[0, 0.2[","[0, 0.2[","[0.18, 1[","[14000, 16000["
1,2685,NaN,NaN,NaN,NaN,NaN,NaN
2,4921,Utah,Suburb,"[0, 0.2[","[0.2, 0.4[","[0.18, 1[","[6000, 8000["
3,3188,NaN,NaN,NaN,NaN,NaN,NaN
4,2238,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
engagmentDF.head()

,time,lp_id,pct_access,engagement_index
0,2020-01-01,93690.0,0.00,NaN
1,2020-01-01,17941.0,0.03,0.90
2,2020-01-01,65358.0,0.03,1.20
3,2020-01-01,98265.0,0.57,37.79
4,2020-01-01,59257.0,0.00,NaN


In [5]:
productInfoDF.head()

,LP ID,URL,Product Name,Provider/Company Name,Sector(s),Primary Essential Function
0,13117,https://www.splashmath.com,SplashLearn,StudyPad Inc.,PreK-12,LC - Digital Learning Platforms
1,66933,https://abcmouse.com,ABCmouse.com,"Age of Learning, Inc",PreK-12,LC - Digital Learning Platforms
2,50479,https://www.abcya.com,ABCya!,"ABCya.com, LLC",PreK-12,"LC - Sites, Resources & Reference - Games & Si..."
3,92993,http://www.aleks.com/,ALEKS,McGraw-Hill PreK-12,PreK-12; Higher Ed,LC - Digital Learning Platforms
4,73104,https://www.achieve3000.com/,Achieve3000,Achieve3000,PreK-12,LC - Digital Learning Platforms


In [6]:
(discritDF.shape)
print(engagmentDF.shape)
print(productInfoDF.shape)


(104003, 4)
(372, 6)


In [7]:
stateDisc = discritDF[["state","district_id"]].groupby(["state"])["district_id"].count().to_frame()

In [8]:
productInfoDF["Sector(s)"].unique()

array(['PreK-12', 'PreK-12; Higher Ed', 'PreK-12; Higher Ed; Corporate',
       nan, 'Corporate', 'Higher Ed; Corporate'], dtype=object)

In [9]:
temp1 = os.path.join(os.getcwd(), "./engagement_data/1000.csv")
engData1000 = pd.read_csv(temp1)

temp2 = os.path.join(os.getcwd(), "./engagement_data/1039.csv")
engData1039 = pd.read_csv(temp2)

temp3 = os.path.join(os.getcwd(), "./engagement_data/1044.csv") 
engData1044 = pd.read_csv(temp3)

In [10]:
len(engData1000["time"].unique())

366

In [11]:
productInfoDF["Primary Essential Function"].unique()
# LC = Learning & Curriculum,
# CM = Classroom Management, 
# and SDO = School & District Operations. 
# Each of these categories have multiple sub-categories 
# with which the products were labeled |

array(['LC - Digital Learning Platforms',
       'LC - Sites, Resources & Reference - Games & Simulations',
       'LC - Courseware & Textbooks', 'LC - Study Tools - Q&A',
       'LC - Sites, Resources & Reference - Streaming Services',
       'LC - Study Tools',
       'CM - Teacher Resources - Professional Learning',
       'SDO - Learning Management Systems (LMS)',
       'LC - Content Creation & Curation',
       'LC - Sites, Resources & Reference',
       'LC - Online Course Providers & Technical Skills Development',
       'CM - Classroom Engagement & Instruction - Communication & Messaging',
       'SDO - School Management Software - SSO',
       'LC - Sites, Resources & Reference - Thesaurus & Dictionary',
       'LC/CM/SDO - Other', 'SDO - Data, Analytics & Reporting',
       'LC - Sites, Resources & Reference - Encyclopedia',
       'CM - Classroom Engagement & Instruction - Classroom Management',
       'LC - Sites, Resources & Reference - Digital Collection & Repository',
 

In [12]:
temp4 = os.path.join(os.getcwd(), "./engagement_data/8815.csv") 
engData8815 = pd.read_csv(temp4)
len(engData8815["time"].unique())

312

In [13]:
DennisDF = pd.read_csv(os.path.join(os.getcwd(), "./ACSST1Y2021.S1501-2023-03-08T022240.csv"))

In [14]:
print(DennisDF.columns)

Index(['Label (Grouping)', 'Alabama!!Total!!Estimate',
       'Alabama!!Percent!!Estimate', 'Alabama!!Male!!Estimate',
       'Alabama!!Percent Male!!Estimate', 'Alabama!!Female!!Estimate',
       'Alabama!!Percent Female!!Estimate', 'Alaska!!Total!!Estimate',
       'Alaska!!Percent!!Estimate', 'Alaska!!Male!!Estimate',
       ...
       'Wyoming!!Male!!Estimate', 'Wyoming!!Percent Male!!Estimate',
       'Wyoming!!Female!!Estimate', 'Wyoming!!Percent Female!!Estimate',
       'Puerto Rico!!Total!!Estimate', 'Puerto Rico!!Percent!!Estimate',
       'Puerto Rico!!Male!!Estimate', 'Puerto Rico!!Percent Male!!Estimate',
       'Puerto Rico!!Female!!Estimate',
       'Puerto Rico!!Percent Female!!Estimate'],
      dtype='object', length=313)


In [15]:
# discritDF[["locale","pct_free/reduced","state"]].groupby(["locale","pct_free/reduced"]).count()
discritDF.groupby(["state","pct_free/reduced"]).count()

district_id  locale  pct_black/hispanic  \
state          pct_free/reduced                                            
California     [0, 0.2[                    5       5                   5   
               [0.2, 0.4[                  3       3                   3   
               [0.4, 0.6[                  1       1                   1   
               [0.6, 0.8[                  3       3                   3   
Connecticut    [0, 0.2[                   12      12                  12   
               [0.2, 0.4[                 11      11                  11   
               [0.4, 0.6[                  7       7                   7   
Florida        [0.4, 0.6[                  1       1                   1   
Illinois       [0, 0.2[                    8       8                   8   
               [0.2, 0.4[                  3       3                   3   
               [0.4, 0.6[                  3       3                   3   
               [0.6, 0.8[                  2       2                   2   
               [0.8, 1[                    2       2                   2   
Indiana        [0.4, 0.6[                  4       4                   4   
               [0.6, 0.8[                  2       2                   2   
               [0.8, 1[                    1       1                   1   
Michigan       [0.4, 0.6[                  1       1                   1   
               [0.6, 0.8[                  1       1                   1   
Minnesota      [0.6, 0.8[                  1       1                   1   
Missouri       [0, 0.2[                    1       1                   1   
               [0.2, 0.4[                  4       4                   4   
               [0.4, 0.6[                  1       1                   1   
New Hampshire  [0.2, 0.4[                  2       2                   2   
New Jersey     [0, 0.2[                    2       2                   2   
New York       [0.2, 0.4[                  5       5                   5   
               [0.4, 0.6[                  2       2                   2   
               [0.8, 1[                    1       1                   1   
North Carolina [0.2, 0.4[                  2       2                   2   
               [0.6, 0.8[                  2       2                   2   
North Dakota   [0, 0.2[                    1       1                   1   
Ohio           [0, 0.2[                    7       7                   7   
               [0.2, 0.4[                  2       2                   2   
               [0.4, 0.6[                  1       1                   1   
Texas          [0.4, 0.6[                  2       2                   2   
Utah           [0, 0.2[                    7       7                   7   
               [0.2, 0.4[                 13      13                  13   
               [0.4, 0.6[                  9       9                   9   
Virginia       [0.2, 0.4[                  1       1                   1   
               [0.4, 0.6[                  2       2                   2   
               [0.6, 0.8[                  1       1                   1   
Washington     [0, 0.2[                    1       1                   1   
               [0.2, 0.4[                  1       1                   1   
               [0.4, 0.6[                  3       3                   3   
               [0.6, 0.8[                  1       1                   1   
Wisconsin      [0, 0.2[                    2       2                   2   
               [0.2, 0.4[                  1       1                   1   

                                 county_connections_ratio  pp_total_raw  
state          pct_free/reduced                                          
California     [0, 0.2[                                 5             0  
               [0.2, 0.4[                               3             0  
               [0.4, 0.6[                               1             0  
              

Engagement_data 
<p1>The engagement data are aggregated at school district level, and each file in the folder `engagement_data` represents data from one school district. The 4-digit file name represents `district_id` which can be used to link to district information in `district_info.csv`. The `lp_id` can be used to link to product information in `product_info.csv`.
</p1>